<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EC%A6%9D%EB%B6%84/B_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A6%9D%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		'# df_G_inc.csv'  'My Drive'	    데이터.zip
'Colab Notebooks'	 G_inc_model.pt    P_inc_model.pt
'답안 작성용 파일.csv'	 I_ori_model.pt    S_ori_model.pt


In [3]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [12]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [5]:
df=df[(df['digit_1'] =='B')] #조건에 맞는 데이터 출력

In [6]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000332,B,7,72,염전에서,바닷물을 태양열에 증발시켜,소금 생산
1,id_0000594,B,7,72,염전에서,바닷물을 태양열에 증발시켜,천일염 생산
2,id_0000869,B,7,71,"ㅌ,사석",채취및 사석,"골재,석재 제조"
3,id_0000873,B,7,72,염전에서,바닷물을 태양열에 증발시켜,천일염 생산
4,id_0001037,B,7,72,염전에서,바닷물을 태양열에 증발시켜,천일염 생산
...,...,...,...,...,...,...,...
419,id_0997123,B,7,71,원석을채취하여,"폐석투입, 파쇄",쇄석(골재)
420,id_0997502,B,7,71,원석,"입고, 분쇄",골재
421,id_0997922,B,7,71,원석자갈,"원석자갈, 파쇄분쇄",자갈
422,id_0998307,B,7,72,염전에서,바닷물을 태양열로 증발시켜,천일염 생산


In [7]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000332,B,7,72,염전에서 바닷물을 태양열에 증발시켜 소금 생산
1,id_0000594,B,7,72,염전에서 바닷물을 태양열에 증발시켜 천일염 생산
2,id_0000869,B,7,71,"ㅌ,사석 채취및 사석 골재,석재 제조"
3,id_0000873,B,7,72,염전에서 바닷물을 태양열에 증발시켜 천일염 생산
4,id_0001037,B,7,72,염전에서 바닷물을 태양열에 증발시켜 천일염 생산
...,...,...,...,...,...
419,id_0997123,B,7,71,"원석을채취하여 폐석투입, 파쇄 쇄석(골재)"
420,id_0997502,B,7,71,"원석 입고, 분쇄 골재"
421,id_0997922,B,7,71,"원석자갈 원석자갈, 파쇄분쇄 자갈"
422,id_0998307,B,7,72,염전에서 바닷물을 태양열로 증발시켜 천일염 생산


In [8]:
df['digit_3'].value_counts()

72    223
71    190
62      6
80      3
61      1
51      1
Name: digit_3, dtype: int64

## **# 데이터 증분 및 맞춤법 교정**

In [ ]:
# 맞춤법 검사

In [9]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-sbqr8jtb
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-sbqr8jtb
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=36518fef6475d3a2b758e405a42ba90ccc159bd2f03d5377a724e42e090397cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-zl8p6u46/wheels/31/c2/c7/ddbbd9a84ddb0a67c2e52e3324d4b7b47dcf9581af9cd784de
Successfully built py-hanspell


In [10]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [11]:
df.to_csv('df_B_spellcheck.csv', index=False, encoding='utf-8-sig')

In [ ]:
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000006,C,29,291,철 절삭. 용접 카프라 배관자재
1,id_0000008,C,10,107,쌀을 가지고 가공하여 떡 제조
2,id_0000011,C,10,102,"멸치 입고, 가공"
3,id_0000021,C,10,107,쌀을 가지고 가루로 분쇄하여 떡을 쪄서 백설기 꿀떡 절편 판매
4,id_0000027,C,22,221,"고무 절단가공 개스킷, 다이아 프레임"
...,...,...,...,...,...
105187,id_0999971,C,28,281,"트랜스, 다이오드, 칩 조립 전원공급장치(파워 서플라이)"
105188,id_0999976,C,25,259,기타 임가공
105189,id_0999986,C,10,106,벼를 가지고 벼를 도정하여 소비자에게 판매
105190,id_0999994,C,32,320,"카본 열선, 퀼팅 구조체 조립. 배열. 봉제하여 의료용 매트리스"


In [ ]:
df['digit_3'].value_counts()

912    15468
911     9779
902     3039
901     1465
Name: digit_3, dtype: int64

In [ ]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [ ]:
df1=df[df['digit_3']==901] 
df2=df[df['digit_3']==902] 
df3=df[df['digit_3']==911] 
df4=df[df['digit_3']==912] 

df100=pd.concat([df1, df2])
df200=pd.concat([df3, df4])

df100=df100.reset_index(drop=True) #*5
df200=df200.reset_index(drop=True) #*0

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df100['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=4)
  id_list=['p'+str(df100['AI_id'][n]), 'p'+str(df100['AI_id'][n]), 'p'+str(df100['AI_id'][n]), 'p'+str(df100['AI_id'][n]),'p'+str(df100['AI_id'][n])]
  digit_1_list=[df100['digit_1'][n], df100['digit_1'][n], df100['digit_1'][n], df100['digit_1'][n],df100['digit_1'][n]]
  digit_2_list=[df100['digit_2'][n], df100['digit_2'][n], df100['digit_2'][n], df100['digit_2'][n],df100['digit_2'][n]]
  digit_3_list=[df100['digit_3'][n], df100['digit_3'][n], df100['digit_3'][n], df100['digit_3'][n],df100['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df100_to_insert = pd.DataFrame(data_to_insert)
  df100= df100.append(df100_to_insert, ignore_index=True)
  n=n+1

기존 데이터 합치기

In [ ]:
df=pd.concat([df100, df200])
df.reset_index(drop=True)

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000144,R,90,901,"사업장에서 공연, 연주 기획"
1,id_0000449,R,90,901,"공연, 예술단체에서 일반인들에게 무용, 음악예술행위 서비스"
2,id_0000503,R,90,901,사업장에서 공연 예술
3,id_0001922,R,90,901,사업장에서 공연예술행사 공연제작
4,id_0001925,R,90,901,"사업장에서 공연예술행사 작품 제작, 시연"
...,...,...,...,...,...
52266,id_0999826,R,91,912,노래연습장 노래연습기를 갖추고 노래연습
52267,id_0999850,R,91,912,노래방에서 시설을 갖추고 노래방 운영
52268,id_0999854,R,91,912,노래방에서 노래기기 시설을 갖추고 노래연습
52269,id_0999886,R,91,912,노래연습장 노래연습 서비스 제공 노래방 기기 대여


In [ ]:
df['digit_3'].value_counts()

902    18234
912    15468
911     9779
901     8790
Name: digit_3, dtype: int64

In [ ]:
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000144,R,90,901,"사업장에서 공연, 연주 기획"
1,id_0000449,R,90,901,"공연, 예술단체에서 일반인들에게 무용, 음악예술행위 서비스"
2,id_0000503,R,90,901,사업장에서 공연 예술
3,id_0001922,R,90,901,사업장에서 공연예술행사 공연제작
4,id_0001925,R,90,901,"사업장에서 공연예술행사 작품 제작, 시연"
...,...,...,...,...,...
25242,id_0999826,R,91,912,노래연습장 노래연습기를 갖추고 노래연습
25243,id_0999850,R,91,912,노래방에서 시설을 갖추고 노래방 운영
25244,id_0999854,R,91,912,노래방에서 노래기기 시설을 갖추고 노래연습
25245,id_0999886,R,91,912,노래연습장 노래연습 서비스 제공 노래방 기기 대여


In [ ]:
df.to_csv('df_R_inc.csv', index=False, encoding='utf-8-sig')